<a href="https://colab.research.google.com/github/Nishaan-Ghimire/A2/blob/main/A2_v1_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d ricardosalvatorelli1/paddydoctor -p /content/drive/MyDrive/KaggleDatasets --unzip


Dataset URL: https://www.kaggle.com/datasets/ricardosalvatorelli1/paddydoctor
License(s): unknown
User cancelled operation


In [54]:
import os
import json
import logging
import tensorflow as tf
from tensorflow.keras.applications import (
    EfficientNetV2B3, EfficientNetV2M, EfficientNetV2L,
    ConvNeXtBase, ConvNeXtLarge,
    ResNet152V2, DenseNet201,
    InceptionResNetV2
)
from tensorflow.keras.layers import (
    Dense, Dropout, GlobalAveragePooling2D, BatchNormalization,
    MultiHeadAttention, LayerNormalization, Add, Reshape, Permute,
    Conv2D, DepthwiseConv2D, Activation
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import AdamW, Adam
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau, EarlyStopping, ModelCheckpoint,
    TensorBoard, CSVLogger
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.mixed_precision import set_global_policy
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [55]:

try:
    import cv2
except ImportError:
    print("📦 Installing opencv...")
    os.system("pip install opencv-python")
    import cv2

# === T4 GPU COLAB OPTIMIZED CONFIG 🔥 ===
# Maxed out for free T4 GPU while being memory-safe
IMG_SIZE = 384      # Sweet spot for T4 - big enough to be compute heavy
BATCH_SIZE = 6      # Optimized for T4's 16GB VRAM
EPOCHS = 100        # Enough to get good results
TRAIN_DATA_DIR = "/content/drive/MyDrive/KaggleDatasets/train_images/"
TEST_DATA_DIR = "/content/drive/MyDrive/KaggleDatasets/test_images/"
NUM_CLASSES = 10
MODELS_TO_ENSEMBLE = 3  # Train multiple models for ensemble

# Enable mixed precision for MAXIMUM SPEED 🚀
set_global_policy('mixed_float16')

# Setup logging with cool emojis
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)


In [56]:

class HeavyAugmentation:
    """Augmentation pipeline that'll make your data unrecognizable 😈"""

    def __init__(self, img_size=384):
        self.img_size = img_size
        self.augment = A.Compose([
            # Geometric transforms
            A.RandomResizedCrop(img_size, img_size, scale=(0.6, 1.0), p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.RandomRotate90(p=0.5),
            A.Rotate(limit=30, p=0.7),
            A.ShiftScaleRotate(
                shift_limit=0.2,
                scale_limit=0.2,
                rotate_limit=30,
                p=0.8
            ),

            # Color augmentations - make it spicy 🌶️
            A.RandomBrightnessContrast(
                brightness_limit=0.3,
                contrast_limit=0.3,
                p=0.8
            ),
            A.HueSaturationValue(
                hue_shift_limit=20,
                sat_shift_limit=30,
                val_shift_limit=20,
                p=0.8
            ),
            A.RGBShift(
                r_shift_limit=20,
                g_shift_limit=20,
                b_shift_limit=20,
                p=0.7
            ),
            A.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.2,
                hue=0.1,
                p=0.7
            ),

            # Noise and blur - chaos mode activated 💀
            A.OneOf([
                A.GaussNoise(var_limit=(10, 50)),
                A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5)),
            ], p=0.6),

            A.OneOf([
                A.GaussianBlur(blur_limit=5),
                A.MotionBlur(blur_limit=5),
                A.MedianBlur(blur_limit=3),
            ], p=0.4),

            # Geometric distortions
            A.OpticalDistortion(distort_limit=0.2, shift_limit=0.2, p=0.4),
            A.GridDistortion(num_steps=5, distort_limit=0.2, p=0.4),
            A.ElasticTransform(
                alpha=120,
                sigma=120 * 0.05,
                alpha_affine=120 * 0.03,
                p=0.4
            ),

            # Cutout variants - randomly remove patches
            A.CoarseDropout(
                max_holes=8,
                max_height=24,
                max_width=24,
                fill_value=0,
                p=0.6
            ),
            A.Cutout(
                num_holes=12,
                max_h_size=24,
                max_w_size=24,
                fill_value=0,
                p=0.5
            ),

            # Weather effects for extra spice
            A.RandomShadow(p=0.3),
            A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.2, p=0.2),
            A.RandomSunFlare(p=0.1),
            A.RandomRain(p=0.1),

            # Normalize at the end
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __call__(self, image):
        if isinstance(image, np.ndarray):
            return self.augment(image=image)['image']
        return image


In [57]:

class CustomTransformerBlock:
    """Custom transformer layers because we're fancy like that ✨"""

    @staticmethod
    def attention_block(x, num_heads=12, key_dim=64, dropout_rate=0.1):
        # Multi-head self-attention
        attn_output = MultiHeadAttention(
            num_heads=num_heads,
            key_dim=key_dim,
            dropout=dropout_rate,
            name=f'mha_{np.random.randint(1000)}'
        )(x, x)

        # Add & Norm
        x = Add()([x, attn_output])
        x = LayerNormalization()(x)

        # Feed Forward Network
        ffn = Sequential([
            Dense(x.shape[-1] * 4, activation='gelu'),
            Dropout(dropout_rate),
            Dense(x.shape[-1])
        ])

        ffn_output = ffn(x)
        x = Add()([x, ffn_output])
        x = LayerNormalization()(x)

        return x


In [58]:

class T4ModelBuilder:
    """Build models that'll make your T4 GPU work hard but not crash 🔥"""

    def __init__(self, num_classes, img_size=384):
        self.num_classes = num_classes
        self.img_size = img_size

    def build_efficientnet_v2_beast(self):
        """EfficientNetV2-L with custom head - The Big Boss 👑"""
        logger.info("🚀 Building EfficientNetV2-L Beast Model")

        base_model = EfficientNetV2L(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_size, self.img_size, 3)
        )

        # Unfreeze last 80 layers for fine-tuning
        for layer in base_model.layers[-80:]:
            layer.trainable = True

        x = base_model.output
        x = GlobalAveragePooling2D(name='global_avg_pool')(x)

        # Transform for attention
        x = Dense(768, name='transformer_proj')(x)
        x = Reshape((1, 768))(x)

        # Add transformer blocks
        x = CustomTransformerBlock.attention_block(x, num_heads=12, key_dim=64)
        x = CustomTransformerBlock.attention_block(x, num_heads=8, key_dim=96)

        x = tf.keras.layers.Flatten()(x)

        # Dense layers with different activations
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(2048, activation='gelu', name='dense_2048')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.4)(x)
        x = Dense(1024, activation='swish', name='dense_1024')(x)
        x = Dropout(0.3)(x)
        x = Dense(512, activation='mish', name='dense_512')(x)
        x = Dropout(0.2)(x)

        predictions = Dense(
            self.num_classes,
            activation='softmax',
            dtype='float32',
            name='predictions'
        )(x)

        model = Model(inputs=base_model.input, outputs=predictions)
        return model

    def build_convnext_monster(self):
        """ConvNeXt-Large - The Convolution King 👹"""
        logger.info("🔥 Building ConvNeXt Large Monster")

        base_model = ConvNeXtLarge(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_size, self.img_size, 3)
        )

        # Fine-tune the entire model
        base_model.trainable = True

        x = base_model.output
        x = GlobalAveragePooling2D()(x)

        # Multiple dense layers with batch norm
        x = BatchNormalization()(x)
        x = Dropout(0.6)(x)
        x = Dense(3072, activation='gelu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1536, activation='swish')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.4)(x)
        x = Dense(768, activation='mish')(x)
        x = Dropout(0.3)(x)

        predictions = Dense(
            self.num_classes,
            activation='softmax',
            dtype='float32'
        )(x)

        model = Model(inputs=base_model.input, outputs=predictions)
        return model

    def build_inception_resnet_hybrid(self):
        """InceptionResNetV2 with custom attention - The Hybrid Beast 🤖"""
        logger.info("🧠 Building InceptionResNetV2 Hybrid")

        base_model = InceptionResNetV2(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_size, self.img_size, 3)
        )

        # Unfreeze last 100 layers
        for layer in base_model.layers[-100:]:
            layer.trainable = True

        x = base_model.output
        x = GlobalAveragePooling2D()(x)

        # Add some custom convolution magic
        x = Dense(1024)(x)
        x = Reshape((1, 1024))(x)

        # Attention mechanism
        x = CustomTransformerBlock.attention_block(x, num_heads=16, key_dim=64)

        x = tf.keras.layers.Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(2048, activation='gelu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.4)(x)
        x = Dense(1024, activation='swish')(x)
        x = Dropout(0.3)(x)

        predictions = Dense(
            self.num_classes,
            activation='softmax',
            dtype='float32'
        )(x)

        model = Model(inputs=base_model.input, outputs=predictions)
        return model

    def build_densenet_enhanced(self):
        """DenseNet201 Enhanced - Dense but Smart 🧮"""
        logger.info("📊 Building Enhanced DenseNet201")

        base_model = DenseNet201(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_size, self.img_size, 3)
        )

        # Fine-tune last layers
        for layer in base_model.layers[-60:]:
            layer.trainable = True

        x = base_model.output
        x = GlobalAveragePooling2D()(x)

        # Enhanced dense layers
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(2048, activation='gelu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.4)(x)
        x = Dense(1024, activation='swish')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        x = Dense(512, activation='mish')(x)
        x = Dropout(0.2)(x)

        predictions = Dense(
            self.num_classes,
            activation='softmax',
            dtype='float32'
        )(x)

        model = Model(inputs=base_model.input, outputs=predictions)
        return model


In [63]:

class T4Trainer:
    """Training pipeline optimized for T4 GPU 🎯"""

    def __init__(self, model, model_name, data_dir, img_size=384, batch_size=6):
        self.model = model
        self.model_name = model_name
        self.data_dir = data_dir
        self.img_size = img_size
        self.batch_size = batch_size
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    def setup_data_generators(self):
        """Setup data generators with INSANE augmentation 🌪️"""

        # Custom preprocessing function
        def preprocess_input(x):
            return HeavyAugmentation(self.img_size)(x)

        # Training generator
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=0.2,
            preprocessing_function=preprocess_input if hasattr(self, 'use_heavy_aug') else None
        )

        # Validation generator (no augmentation)
        val_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=0.2
        )

        self.train_gen = train_datagen.flow_from_directory(
            self.data_dir,
            target_size=(self.img_size, self.img_size),
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='training',
            shuffle=True
        )

        self.val_gen = val_datagen.flow_from_directory(
            self.data_dir,
            target_size=(self.img_size, self.img_size),
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='validation',
            shuffle=False
        )

        logger.info(f"📊 Training samples: {self.train_gen.samples}")
        logger.info(f"📊 Validation samples: {self.val_gen.samples}")

    def compile_model(self):
        """Compile with advanced optimizer 🚀"""

        # Calculate steps for learning rate schedule
        steps_per_epoch = self.train_gen.samples // self.batch_size
        total_steps = steps_per_epoch * 80  # 80 epochs for schedule

        # Cosine decay learning rate
        lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
            initial_learning_rate=2e-4,
            decay_steps=total_steps,
            alpha=1e-6
        )

        # AdamW optimizer for better generalization
        optimizer = AdamW(
            learning_rate=lr_schedule,
            weight_decay=1e-4,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-7
        )

        self.model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy', 'top_2_categorical_accuracy']
        )

        logger.info("✅ Model compiled with AdamW optimizer")

    def setup_callbacks(self):
        """Callbacks for monitoring and saving 📈"""

        callbacks = [
            ModelCheckpoint(
                f'best_{self.model_name}_{self.timestamp}.h5',
                monitor='val_accuracy',
                save_best_only=True,
                save_weights_only=False,
                verbose=1,
                mode='max'
            ),

            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.3,
                patience=7,
                min_lr=1e-7,
                verbose=1,
                cooldown=3
            ),

            EarlyStopping(
                monitor='val_accuracy',
                patience=15,
                restore_best_weights=True,
                verbose=1,
                mode='max',
                min_delta=0.001
            ),

            CSVLogger(
                f'training_log_{self.model_name}_{self.timestamp}.csv',
                append=True
            )
        ]

        return callbacks

    def train(self, epochs=100):
        """The training that'll make your T4 GPU sweat 💪"""

        logger.info(f"🚀 Starting training for {self.model_name}")
        logger.info("Get ready for some serious GPU workout! 🔥")

        self.setup_data_generators()
        self.compile_model()
        callbacks = self.setup_callbacks()

        # Print model info
        total_params = self.model.count_params()
        trainable_params = sum([tf.keras.backend.count_params(w) for w in self.model.trainable_weights])

        print(f"📊 Total parameters: {total_params:,}")
        print(f"📊 Trainable parameters: {trainable_params:,}")
        print(f"🎯 Image size: {self.img_size}x{self.img_size}")
        print(f"📦 Batch size: {self.batch_size}")

        history = self.model.fit(
            self.train_gen,
            validation_data=self.val_gen,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1
        )

        return history

    def evaluate_model(self):
        """Comprehensive evaluation with plots 📊"""

        logger.info("📊 Evaluating model performance")

        # Reset generator
        self.val_gen.reset()

        # Get predictions
        predictions = self.model.predict(self.val_gen, verbose=1)
        y_pred = np.argmax(predictions, axis=1)
        y_true = self.val_gen.classes

        # Get class names
        class_names = list(self.val_gen.class_indices.keys())

        # Classification report
        print("\n" + "="*60)
        print("🎯 CLASSIFICATION REPORT")
        print("="*60)
        print(classification_report(y_true, y_pred, target_names=class_names))

        # Confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(12, 10))
        sns.heatmap(
            cm,
            annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names
        )
        plt.title(f'Confusion Matrix - {self.model_name}', fontsize=16)
        plt.ylabel('True Label', fontsize=12)
        plt.xlabel('Predicted Label', fontsize=12)
        plt.tight_layout()
        plt.savefig(f'confusion_matrix_{self.model_name}_{self.timestamp}.png', dpi=300)
        plt.show()

        # Calculate accuracy
        accuracy = np.sum(y_pred == y_true) / len(y_true)
        print(f"\n🎯 Final Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")


In [64]:

def setup_gpu_memory():
    """Configure GPU memory for T4 optimization 🎮"""

    physical_devices = tf.config.list_physical_devices('GPU')

    if physical_devices:
        try:
            # Enable memory growth
            tf.config.experimental.set_memory_growth(physical_devices[0], True)

            # Optional: Set memory limit (uncomment if you get OOM errors)
            # tf.config.experimental.set_virtual_device_configuration(
            #     physical_devices[0],
            #     [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=14000)]
            # )

            print(f"🎮 GPU configured: {physical_devices[0]}")
            print("💚 Memory growth enabled for T4 optimization")

        except RuntimeError as e:
            print(f"⚠️ GPU configuration error: {e}")
    else:
        print("❌ No GPU found! This will be slow on CPU")


In [65]:

def main():
    """Main function to run the T4-optimized pipeline 🚀"""

    print("🔥" * 20)
    print("T4 GPU OPTIMIZED TRAINING PIPELINE")
    print("Free Colab Edition - Max Performance! 💪")
    print("🔥" * 20)

    # Setup GPU
    setup_gpu_memory()

    # Initialize model builder
    builder = T4ModelBuilder(num_classes=NUM_CLASSES, img_size=IMG_SIZE)

    # Models to train (all Colab compatible)
    models_to_train = [
        ('EfficientNetV2L_Beast', builder.build_efficientnet_v2_beast),
        ('ConvNeXt_Large_Monster', builder.build_convnext_monster),
        ('InceptionResNetV2_Hybrid', builder.build_inception_resnet_hybrid),
        ('DenseNet201_Enhanced', builder.build_densenet_enhanced),
    ]

    trained_models = {}

    for i, (model_name, model_builder) in enumerate(models_to_train):
        print(f"\n🚀 Training Model {i+1}/{len(models_to_train)}: {model_name}")
        print("Time to make that T4 work! 💪")

        try:
            # Build model
            model = model_builder()

            # Train model
            trainer = T4Trainer(model, model_name, TRAIN_DATA_DIR, IMG_SIZE, BATCH_SIZE)
            history = trainer.train(epochs=EPOCHS)

            # Evaluate
            trainer.evaluate_model()

            trained_models[model_name] = {
                'model': model,
                'history': history,
                'trainer': trainer
            }

            print(f"✅ {model_name} training completed!")

            # Clear memory after each model (important for Colab)
            tf.keras.backend.clear_session()

        except Exception as e:
            logger.error(f"❌ Failed to train {model_name}: {str(e)}")
            tf.keras.backend.clear_session()
            continue

    print("\n🎉 T4 TRAINING PIPELINE COMPLETE! 🎉")
    print("Your free T4 GPU just got a serious workout! 💪")
    print(f"Successfully trained {len(trained_models)} models 🔥")


In [66]:
if __name__ == "__main__":
    main()

🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥
T4 GPU OPTIMIZED TRAINING PIPELINE
Free Colab Edition - Max Performance! 💪
🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥
🎮 GPU configured: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
💚 Memory growth enabled for T4 optimization

🚀 Training Model 1/4: EfficientNetV2L_Beast
Time to make that T4 work! 💪
Found 8330 images belonging to 10 classes.
Found 2077 images belonging to 10 classes.
📊 Total parameters: 137,120,682
📊 Trainable parameters: 136,602,474
🎯 Image size: 384x384
📦 Batch size: 6


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 12, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
ERROR:__main__:❌ Failed to train EfficientNetV2L_Beast: Could not interpret metric identifier: top_2_categorical_accuracy



🚀 Training Model 2/4: ConvNeXt_Large_Monster
Time to make that T4 work! 💪
Found 8330 images belonging to 10 classes.
Found 2077 images belonging to 10 classes.
📊 Total parameters: 206,884,810
📊 Trainable parameters: 206,872,522
🎯 Image size: 384x384
📦 Batch size: 6
Epoch 1/100


...


🚀 Training Model 3/4: InceptionResNetV2_Hybrid
Time to make that T4 work! 💪
Found 8330 images belonging to 10 classes.
Found 2077 images belonging to 10 classes.
📊 Total parameters: 72,726,762
📊 Trainable parameters: 72,660,074
🎯 Image size: 384x384
📦 Batch size: 6
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 16, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
ERROR:__main__:❌ Failed to train InceptionResNetV2_Hybrid: Could not interpret metric identifier: top_2_categorical_accuracy



🚀 Training Model 4/4: DenseNet201_Enhanced
Time to make that T4 work! 💪
Found 8330 images belonging to 10 classes.
Found 2077 images belonging to 10 classes.
📊 Total parameters: 24,904,266
📊 Trainable parameters: 24,665,226
🎯 Image size: 384x384
📦 Batch size: 6
Epoch 1/100


ERROR:__main__:❌ Failed to train DenseNet201_Enhanced: Could not interpret metric identifier: top_2_categorical_accuracy



🎉 T4 TRAINING PIPELINE COMPLETE! 🎉
Your free T4 GPU just got a serious workout! 💪
Successfully trained 0 models 🔥
